# Gitex 2023 - Charmed Spark

Check that Spark is running

In [1]:
sc

<SparkContext master=k8s://https://192.168.1.4:16443 appName=PySparkShell>

In [2]:
# Optional 
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


## Case Description

You are the CEO of a consumer goods reseller company and one morning you receive a strange phone call. On the other side of the line, there is a detective from the Interpol that needs your help to solve a crime. There has been a murder in Leuven, Belgium, but the killer in the rush of running away has forgotten a suitcase. Unfortunately, the suitcase did not bring any ID or the name of its owner. However, in the suitcase there were found some documents that may help to identify the killer. Among the documents there is in fact the confirmation of a successful bank transfer to your company of  332.68 $, that has therefore suggested the Interpol that the killer may be one of your customers. Moreover, the suitcases also contains some extra material:

1. Various receipts from bars, restaurants and shops in Sidney.
2. There are several pictures of what appears to be a trip in the Amazonian jungle. The pictures have dates ranging from 2011-03-01 and 2011-05-01.

The detective hopes that with this information, you may be able to identify the killer among your customers using the track record of your sales. Could you help him?

## Dataset

In [3]:
S3_BUCKET="charmed-spark-gitex-demo"

In [4]:
df = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load(f"s3a://{S3_BUCKET}/data/interpol_case.csv.gz")

In [5]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [6]:
df.show(truncate=True)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

### Analysis

In [7]:
from pyspark.sql.functions import col

In [8]:
quantity_by_stock_code = df.groupBy("StockCode").sum("Quantity")

quantity_by_stock_code

DataFrame[StockCode: string, sum(Quantity): bigint]

In [9]:
top_sellers = quantity_by_stock_code \
        .sort(col("sum(Quantity)").desc()) \
        .limit(10) \
        .toPandas().set_index("StockCode")

top_sellers

,sum(Quantity)
StockCode,
22197,56450
84077,53847
85099B,47363
85123A,38830
84879,36221
21212,36039
23084,30646
22492,26437
22616,26315
